## Run Once Per Notebook Restart (if files still exist)

In [ ]:
import os
import time
from bark_infinity import config
import numpy as np

logger = config.logger
logger.setLevel("WARNING")

from bark_infinity import generation
from bark_infinity import api

import rich
from rich import print
from rich import pretty
from rich.pretty import pprint
from rich import inspect

from pydub import AudioSegment
import ipywidgets as widgets
from IPython.display import display, Audio
from io import BytesIO

# None of this code, just fiddlign with Colab stuff
# Just to save Colab with outputs and float32 wavs are GIGANTO
# actually this doesn't work, the iPython widget converts it back to float32? or I messed up

def display_audio_int16_but(audio_arr_segments, file_name, sample_rate=generation.SAMPLE_RATE,  width='200px'):
    file_name_label = widgets.Label(value=f"Playing: {file_name}")
    file_name_label.layout.width = width
    audio_data_int16 = audio_arr_segments
    if isinstance(audio_data_int16, list):
        audio_data_int16 = np.concatenate(audio_data_int16)        

    #audio_data_int16 = np.int16(audio_data_int16 * np.iinfo(np.int16).max)


    audio_widget = Audio(audio_data_int16, rate=sample_rate)
    display(file_name_label, audio_widget)
    

def on_button_click(button):
    audio_data, sample_rate = librosa.load(button.wav_path, sr=None)
    file_name = os.path.basename(button.wav_path)
    display_audio_int16_but(audio_data,file_name, sample_rate)


def display_wav_files(directory):
    subdirs, wav_files = [], []
    
    for item in os.listdir(directory):
        item_path = os.path.join(directory, item)
        
        if os.path.isfile(item_path) and item_path.endswith('.wav'):
            wav_files.append(item_path)
        elif os.path.isdir(item_path):
            subdirs.append(item_path)

    wav_files.sort(key=lambda x: os.path.basename(x))

    for wav_file in wav_files:

        filename = os.path.basename(wav_file)
        print(f" {filename}")
        button = widgets.Button(description=f"Play {filename}")
        button.wav_path = wav_file  
        button.on_click(on_button_click)
        display(button)

    for subdir in sorted(subdirs):
        print(f"<{subdir}>")
        display_wav_files(subdir)



In [ ]:
text = """With me, brave the tumultuous seas, claim treasures untold, and send foes to their watery 
doom"""
logger.setLevel("DEBUG")
kwargs["semantic_min_eos_p"] = 0.2 
kwargs["text_prompt"] = text
generation.set_seed(37)
kwargs['output_full']  = True
kwargs['output_dir'] = "bark_test"
kwargs["history_prompt"] = None
#kwargs['history_prompt'] = 'custom_speakers/en_fiery.npz'
barkifull, audio_arr_segments_barki = api.generate_audio_barki(text, **kwargs)

In [ ]:
api.gradio_try_to_cancel = False

In [ ]:
kwargs['output_dir'] = "long"
generation.set_seed(37)
full_generation_segments, audio_arr_segments, final_filename_will_be = api.generate_audio_long(**kwargs)

In [ ]:
from bark import SAMPLE_RATE, generate_audio, preload_models, set_seed
from scipy.io.wavfile import write as write_wav
import numpy as np
import datetime

preload_models()

# we actually DO need CUBLAS_WORKSPACE_CONFIG and use_deterministic_algorithms in set_seed, or the wavs are slightly different
# pretty big speed hit to generation though. 1.8it/s to 1.2it/s !

date_str = datetime.datetime.now().strftime("%y-%m%d-%H%M-%S")  
seed = 123

filename = f"seed_test_{date_str}_seed_{seed}.wav"

prompt = "I have a silky smooth voice."

set_seed(seed)

audio_array_1 = generate_audio(prompt)
write_wav(f"{filename}_1.wav", SAMPLE_RATE, audio_array_1)

set_seed(seed)

audio_array_2 = generate_audio(prompt)
write_wav(f"{filename}_2.wav", SAMPLE_RATE, audio_array_2)

assert(np.array_equal(audio_array_1, audio_array_2))


seed = -1

set_seed(seed)
filename = f"seed_test_{date_str}_seed_{seed}.wav"

audio_array_3 = generate_audio(prompt)

write_wav(f"{filename}_3.wav", SAMPLE_RATE, audio_array_3)